In [ ]:
import pandas as pd
import finterstellar as fs

데이터 로딩

In [ ]:
path = './data/'
cd = 'KOSPI 200'
#cd = 'S&P 500'

In [ ]:
# define portfolio universe
portfolio = {
    'World indices' : ['KOSPI 200', 'S&P 500', 'Nikkei 225', 'CSI 300']
}    # 포트폴리오를 딕셔너리 형태로 저장
p_name = 'World indices'    # 포트폴리오 집합 중 분석대상 포트폴리오의 이름 입력
p_cd = portfolio[p_name]    # 포트폴리오 내에서 처리대상 종목코드들을 지정

In [ ]:
ld = fs.LoadData()     # 라이브러리를 ld로 지정

In [ ]:
df = ld.read_master_file(path, p_name)    # ld 모듈의 read_master_file(폴더명, 포트폴리오명)을 이용해 데이터 로딩

RSI 계산 준비

In [ ]:
base_date = '2018-01-02'    # 기준일자 설정

In [ ]:
sample_df = pd.DataFrame()    # 빈 데이터프레임을 생성하고
sample_df[cd] = df[cd].copy()    # 시세를 복사해 넣음
sample_df = sample_df.dropna()     # na를 없애고
sample_df['diff'] = sample_df[cd] - sample_df[cd].shift(1)    # 매일매일 전일과의 주가차이를 계산해서 'diff'라고 저장
sample_df.head(3)

In [ ]:
d, ad, u, au = 0, 0., 0, 0.   # down, accumulated down, up, accumulated up
for i in range(20):    # 기간을 range(일자수) 로 잡아주고 i라는 변수로 순환
    diff = sample_df.shift(i).loc['2018-11-05', 'diff']   # i일 전의 diff값을 읽어옴
    if diff >= 0:
        u += 1
        au += diff
    elif diff < 0:
        d += 1
        ad -= diff
print(u, au, d, ad)

In [ ]:
rsi = au / (au + ad) * 100
rsi

위에서 연습한 내용을 함수로 작성하면

In [ ]:
def rsi(period):
    for p in sample_df.iloc[period:].index:
        d, ad, u, au = 0, 0., 0, 0.    # down, accumulated down, up, accumulated up
        for i in range(period):    # 기간을 range(일자수) 로 잡아주고 i라는 변수로 순환
            diff = sample_df.shift(i).loc[p, 'diff']    # i일 전의 diff값을 읽어옴
            if diff >= 0:    # diff가 +이면
                u += 1    # up 개수 추가
                au += diff    # up 누적값 추가
            elif diff < 0:   # diff가 -이면
                d += 1    # down 개수 추가
                ad -= diff    # down 누적값 추가
        if (au+ad) != 0:
            rsi = au / (au + ad) * 100    # RSI 계산
        else:
            rsi = 50
        
        sample_df.loc[p, 'RSI'+str(period)] = rsi    # 데이터프레임에 RSI(일자) 값 추가

In [ ]:
rsi(5)

In [ ]:
sample_df.tail()

In [ ]:
v = fs.Visualize()      # fs 라이브러리의 Visualize() 모듈을 불러와 v로 지정

In [ ]:
v.multi_line_view(sample_df, base_date, [cd], ['RSI5'], (15,3))
# multi_line_view(데이터프레임, 기준일자, [비교대상1(복수종목)], [비교대상2(복수종목)],(챠트사이즈)) 
# 비교대상1에 굵게 표시할 종목을 입력

In [ ]:
# RSI(20) 값을 계산해 데이터프레임에 추가
rsi(20)

In [ ]:
sample_df.tail()

In [ ]:
v.multi_line_view(sample_df, base_date, [cd], ['RSI20'], (15,3))
# multi_line_view(데이터프레임, 기준일자, [비교대상1(복수종목)], [비교대상2(복수종목)],(챠트사이즈)) 
# 비교대상1에 굵게 표시할 종목을 입력

최근일자에 가중치를 주는 WRSI 계산 함수

In [ ]:
def wrsi(period):
    for p in sample_df.iloc[period:].index:
        d, ad, u, au, multiple = 0, 0., 0, 0., 0.   # down, accumulated down, up, accumulated up
        for i in range(period):    # 기간을 range(일자수) 로 잡아주고 i라는 변수로 순환
            multiple = (period - i) / period   # 최근일자에 가중치
            diff = sample_df.shift(i).loc[p, 'diff']    # i일 전의 diff값을 읽어옴
            if diff >= 0:
                u += 1
                au += diff * multiple    # up 누적값에 multiple을 곱해서 더해줌
            elif diff < 0:
                d += 1
                ad -= diff * multiple    # down 누적값에 multiple을 곱해서 더해줌
        rsi = au / (au + ad) * 100
        
        sample_df.loc[p, 'WRSI'+str(period)] = rsi   # 데이터프레임에 WRSI(일자) 값 추가

WRSI 추세가 꺾였는지 확인하기 위한 추가 factor

In [ ]:
def wris_diff(wris):
    sample_df['wris_diff'] = sample_df[wris] - sample_df[wris].shift(1)

In [ ]:
# WRSI(20) 값을 계산해 데이터프레임에 추가
wrsi(20)

In [ ]:
sample_df.tail()

In [ ]:
# WRSI 추세가 꺾인지 판단하는 wrsi_diff 지표 추가
sample_df['wris_diff'] = sample_df['WRSI20'] - sample_df['WRSI20'].shift(1)

In [ ]:
sample_df.tail()

In [ ]:
v.multi_line_view(sample_df, base_date, [cd], ['WRSI20'], (15,3))
# multi_line_view(데이터프레임, 기준일자, [비교대상1(복수종목)], [비교대상2(복수종목)],(챠트사이즈)) 
# 비교대상1에 굵게 표시할 종목을 입력

In [ ]:
v.multi_line_view(sample_df, base_date, [cd], ['RSI20'], (15,3))

# RSI or WRSI 를 이용해 트레이딩을 한다면?
백테스팅을 해봅시다

In [ ]:
# trend_tradings(데이터프레임, 트레이딩북, 종목코드, 요인1(f1), 지표1(c1), 요인2, 지표2) 함수 정의
def trend_tradings(sample, book, cd, f1, c1, f2, c2):   
    for i in sample.index:   # 데이터프레임을 하나씩 읽어가며
        if sample.loc[i, f1] >= c1:   # 요인1(f1) 값이 지표1(c1)보다 크면
            book.loc[i, 't '+cd] = 'buy'    # buy
            if sample.loc[i, f2] < c2:    # 요인2(f2) 값이 지표2(c2)보다 작으면
                book.loc[i, 't '+cd] = ''    # clear(잔고청산)
        else:    # 위 판단지표에 해당하지 않으면
            book.loc[i, 't '+cd] = ''    # clear
    return (book) 

In [ ]:
sample_df = sample_df.loc[base_date:]    # 기준일자 이후 값으로만 데이터프레임을 채움 (속도를 위해)

In [ ]:
trd = fs.Trade()   # fs 라이브러리의 Trade() 모듈을 불러와 trd로 지정

base_date = trd.check_base_date(sample_df, base_date)   
# check_base_date(데이터프레임, 기준일자) - 지정한 기준일자 데이터가 유효한지 검증
base_date

In [ ]:
book = trd.create_trade_book(sample_df, [cd])
# create_trade_book(데이터프레임, [종목코드]) - 트레이딩북 생성

In [ ]:
book = trend_tradings(sample_df, book, cd, 'WRSI20', 70, 'wris_diff', 0)
# WRSI20 >= 70 이면 buy, wrsi_diff < 0 이면 clear

In [ ]:
book = trd.position(book, [cd])
# position(트레이딩북, 종목코드) - 매매전략에 따른 포지션을 판단해 트레이딩북에 기록
book

In [ ]:
fund_rtn = trd.returns(book, [cd])
# returns(트레이딩북, 종목코드) - 백테스팅 수익률 계산

In [ ]:
bm_rtn = trd.benchmark_return(book, [cd])
# benchmark_return(트레이딩북, 종목코드) - 벤치마크 수익률(전략 미이용시) 계산

In [ ]:
exs_rtn = trd.excess_return(fund_rtn, bm_rtn)
# excess_return(전략수익률, 벤치마크수익률) - 전략의 벤치마크 대비 초과수익률 계산

In [ ]:
book = trend_tradings(sample_df, book, cd, 'RSI20', 60, 'diff', 0)

In [ ]:
book = trd.position(book, [cd])
fund_rtn = trd.returns(book, [cd])

In [ ]:
exs_rtn = trd.excess_return(fund_rtn, bm_rtn)